In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/BX-Users', './input/BX-Books', './input/BX-Book-Ratings']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False, sep=';', encoding = 'ISO-8859-1')

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

# Introduction

**Problem: Book recommendations - „I like Lord of the Rings, what else should I read?“**

Goal:

- take some data, try some approaches, produce some code, get some results

- then come and show us your solution and have a chat around it - show how you think about a specific problem, how you are able to explain what approach you used and why, think about the limitations of the approach and how things could be improved if there was more time, what you think of the results and if they make sense, etc.

Philosophy:

- the actual result and code are not that important – the journey there and potential future paths are more important

- if you are able to follow-through with some ideas then great, if you just start something and have a clear idea on how to proceed that is also useful

- the expectation is that you will spend an evening or two with the task (but there are no bounds to proactivity if you enjoy playing with the problem)

Data: - available open dataset: http://www2.informatik.uni-freiburg.de/~cziegler/BX/

- alternatively feel free to grab any other relevant data set

Tools:

- use whatever you are comfortable in (R, Python, Matlab, Java, SQL,…) or feel free to use it as an opportunity to try out a new language

- it is not a contest in finding the best black-box library and blindly using it – own solutions are preferred even if they are simple

- how to present: up to you – slides, walking through code, drawing, … 

# Load libraries

In [1]:
# Basic
import numpy as np 
# import pandas as pd 
exec(os.environ['IREWR_IMPORTS'])
# ALEX: remove plotting
# import seaborn as sns
# import matplotlib.pyplot as plt 

# # System
# import warnings
# import os
# warnings.filterwarnings("ignore")
# %matplotlib inline

# ALEX: remove path printing
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Load data

In [2]:
users = pd.read_csv('./input/BX-Users.scaled.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1') #encoding = "latin-1"

/tmp/ipykernel_49938/2678319865.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv('./input/BX-Users.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1') #encoding = "latin-1"


In [3]:
books = pd.read_csv('./input/BX-Books.scaled.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1') #encoding = "latin-1

/tmp/ipykernel_49938/1260802686.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('./input/BX-Books.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1') #encoding = "latin-1
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fi

In [4]:
ratings = pd.read_csv('./input/BX-Book-Ratings.scaled.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1')

/tmp/ipykernel_49938/1518381852.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings = pd.read_csv('./input/BX-Book-Ratings.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1')


In [5]:
users.shape

(278858, 3)

In [6]:
books.shape

(271360, 8)

In [7]:
ratings.shape

(1149780, 3)

In [8]:
users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [9]:
ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [10]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [11]:
data = pd.merge(ratings, users, on='User-ID', how='inner')

In [12]:
data = pd.merge(data, books, on='ISBN', how='inner')

In [13]:
# Check
data.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'Location', 'Age', 'Book-Title',
       'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S',
       'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [14]:
#data.rename(columns={'Book-Rating':'BookRating', 'User-ID':'UserID'},inplace=True)

In [15]:
# Drop (TODO: image analysis?)
'''to_drop = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L']

data = data.drop(to_drop, axis=1, inplace=False)'''

"to_drop = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L']\n\ndata = data.drop(to_drop, axis=1, inplace=False)"

In [16]:
data.shape

(1031136, 12)

In [17]:
print('Size of the dataset is: ', data.memory_usage().sum() / 1024**2, ' MB')

Size of the dataset is:  102.270263671875  MB


# EDA

In [18]:
# TODO: EDA in Power BI

In [19]:
data.shape

(1031136, 12)

In [20]:
data.head(5)

User-ID        ISBN  Book-Rating                           Location   Age  \
0   276725  034545104X            0                  tyler, texas, usa   NaN   
1     2313  034545104X            5              cincinnati, ohio, usa  23.0   
2     6543  034545104X            0           strafford, missouri, usa  34.0   
3     8680  034545104X            5  st. charles county, missouri, usa   2.0   
4    10314  034545104X            9             beaverton, oregon, usa   NaN   

             Book-Title Book-Author Year-Of-Publication         Publisher  \
0  Flesh Tones: A Novel  M. J. Rose                2002  Ballantine Books   
1  Flesh Tones: A Novel  M. J. Rose                2002  Ballantine Books   
2  Flesh Tones: A Novel  M. J. Rose                2002  Ballantine Books   
3  Flesh Tones: A Novel  M. J. Rose                2002  Ballantine Books   
4  Flesh Tones: A Novel  M. J. Rose                2002  Ballantine Books   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/034545104X.0...   
1  http://images.amazon.com/images/P/034545104X.0...   
2  http://images.amazon.com/images/P/034545104X.0...   
3  http://images.amazon.com/images/P/034545104X.0...   
4  http://images.amazon.com/images/P/034545104X.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/034545104X.0...   
1  http://images.amazon.com/images/P/034545104X.0...   
2  http://images.amazon.com/images/P/034545104X.0...   
3  http://images.amazon.com/images/P/034545104X.0...   
4  http://images.amazon.com/images/P/034545104X.0...   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/034545104X.0...  
1  http://images.amazon.com/images/P/034545104X.0...  
2  http://images.amazon.com/images/P/034545104X.0...  
3  http://images.amazon.com/images/P/034545104X.0...  
4  http://images.amazon.com/images/P/034545104X.0...

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031136 entries, 0 to 1031135
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   User-ID              1031136 non-null  int64  
 1   ISBN                 1031136 non-null  object 
 2   Book-Rating          1031136 non-null  int64  
 3   Location             1031136 non-null  object 
 4   Age                  753301 non-null   float64
 5   Book-Title           1031136 non-null  object 
 6   Book-Author          1031135 non-null  object 
 7   Year-Of-Publication  1031136 non-null  object 
 8   Publisher            1031134 non-null  object 
 9   Image-URL-S          1031136 non-null  object 
 10  Image-URL-M          1031136 non-null  object 
 11  Image-URL-L          1031132 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 102.3+ MB


In [22]:
print('Number of books: ', data['ISBN'].nunique())

Number of books:  270151


In [23]:
print('Number of users: ',data['User-ID'].nunique())

Number of users:  92106


In [24]:
print('Missing data [%]')
round(data.isnull().sum() / len(data) * 100, 4)

Missing data [%]


User-ID                 0.0000
ISBN                    0.0000
Book-Rating             0.0000
Location                0.0000
Age                    26.9446
Book-Title              0.0000
Book-Author             0.0001
Year-Of-Publication     0.0000
Publisher               0.0002
Image-URL-S             0.0000
Image-URL-M             0.0000
Image-URL-L             0.0004
dtype: float64

In [25]:
# ALEX: remove plotting
# sns.distplot(data['Age'].dropna(), kde=False)
_ = data['Age'].dropna()

In [26]:
print('Number of outliers: ', sum(data['Age'] > 100))

Number of outliers:  2910


In [27]:
# data['Book-Rating'] = data['Book-Rating'].replace(0, None)
data['Book-Rating'] = data['Book-Rating'].replace([0], None)

In [28]:
# ALEX: remove plotting
# sns.countplot(x='Book-Rating', data=data)

In [29]:
print('Average book rating: ', round(data['Book-Rating'].mean(), 2))

Average book rating:  7.63


In [30]:
# Publication by Year
year = pd.to_numeric(data['Year-Of-Publication'], 'coerce').fillna(2099, downcast = 'infer')
# ALEX: remove plotting
# sns.distplot(year, kde=False, hist_kws={"range": [1945,2020]})

In [31]:
country = data['Location'].apply(lambda row: str(row).split(',')[-1])
data.groupby(country)['Book-Rating'].count().sort_values(ascending=False).head(10)

Location
 usa               262464
 canada             35574
 united kingdom     14794
 germany            12151
                    10509
 spain               7197
 australia           6991
 n/a                 6955
 france              4636
 portugal            3445
Name: Book-Rating, dtype: int64

# Feature Engineering

## Change data-type

In [32]:
# Cast to numeric
data['Year-Of-Publication'] = pd.to_numeric(data['Year-Of-Publication'], 'coerce').fillna(2099, downcast = 'infer')

In [33]:
data['Book-Rating'] = data['Book-Rating'].replace(0, None)

## Handle outliers

In [34]:
data['Age'] = np.where(data['Age']>90, None, data['Age'])

## Impute nulls

In [35]:
# Categorical feautes
data[['Book-Author', 'Publisher']] = data[['Book-Author', 'Publisher']].fillna('Unknown')

In [36]:
# Check cat features
data[['Book-Author', 'Publisher']].isnull().sum()

Book-Author    0
Publisher      0
dtype: int64

In [37]:
# Age
median = data["Age"].median()
std = data["Age"].std()
is_null = data["Age"].isnull().sum()
rand_age = np.random.randint(median - std, median + std, size = is_null)
age_slice = data["Age"].copy()
age_slice[pd.isnull(age_slice)] = rand_age
data["Age"] = age_slice
data["Age"] = data["Age"].astype(int)

In [38]:
# Check Age
data['Age'].isnull().sum()

0

## Extract features

In [39]:
data['Country'] = data['Location'].apply(lambda row: str(row).split(',')[-1])

In [40]:
# Drop irelevant
data = data.drop('Location', axis=1)

In [41]:
data['Country'].head()

0     usa
1     usa
2     usa
3     usa
4     usa
Name: Country, dtype: object

### Select English speaking countries 

In [42]:
#TODO: country/language analysis (Babel lib?)

In [43]:
#en_list = ['usa', 'canada', 'united kingdom', 'australia']

In [44]:
#data[data['Country'].isin(en_list)]

# Prepeare dataset

In [45]:
df = data

In [46]:
# Relevant score
df = df[df['Book-Rating'] >= 6]

In [47]:
# Check
df.groupby('ISBN')['User-ID'].count().describe()

count    129014.000000
mean          2.495047
std           7.023557
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         663.000000
Name: User-ID, dtype: float64

In [48]:
df = df.groupby('ISBN').filter(lambda x: len(x) >= 5)

In [49]:
df.groupby('User-ID')['ISBN'].count().describe()

count    41859.000000
mean         3.765642
std         16.091125
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       2450.000000
Name: ISBN, dtype: float64

In [50]:
df = df.groupby('User-ID').filter(lambda x: len(x) >= 5)

In [51]:
df.shape

(105350, 12)

# Simple recommendation

In [52]:
df_p = df.pivot_table(index='ISBN', columns='User-ID', values='Book-Rating')

In [53]:
# Select users who liked LOTR
lotr = df_p.loc['0345339703'] # Lord of the Rings Part 1
like_lotr = lotr[lotr == 10].to_frame().reset_index()
users = like_lotr['User-ID'].to_frame()

In [54]:
# Trim original dataset
liked = pd.merge(users, df, on='User-ID', how='inner')

## Add measures (aggregate) 

In [55]:
rating_count = liked.groupby('ISBN')['Book-Rating'].count().to_frame()

In [56]:
rating_mean = liked.groupby('ISBN')['Book-Rating'].mean().to_frame()

In [57]:
rating_count.rename(columns={'Book-Rating':'Rating-Count'}, inplace=True)

In [58]:
rating_mean.rename(columns={'Book-Rating':'Rating-Mean'}, inplace=True)

In [59]:
liked = pd.merge(liked, rating_count, on='ISBN', how='inner')

In [60]:
liked = pd.merge(liked, rating_mean, on='ISBN', how='inner')

In [61]:
liked['Rating-Mean'] = liked['Rating-Mean'].round(2)

In [62]:
# ALEX: remove plotting
# liked['Rating-Count'].hist()
liked['Rating-Count']

0        1
1        1
2        2
3        2
4       45
        ..
1561     1
1562     1
1563     1
1564     1
1565     1
Name: Rating-Count, Length: 1566, dtype: int64

In [63]:
C = liked['Rating-Mean'].mean()
C

8.89911238825032

In [64]:
m = rating_count.quantile(.995)[0] # .9
m

6.0

In [65]:
# IMDB formula; source: http://trailerpark.weebly.com/imdb-rating.html?source=post_page---------------------------
def weighted_rating(x, m=m, C=C):
    v = x['Rating-Count']
    R = x['Rating-Mean']

    return (v/(v+m) * R) + (m/(m+v) * C)

In [66]:
# Create relevant sub-dataset
liked_q = liked.copy().loc[liked['Rating-Count'] >= m]
liked_q.shape

(134, 14)

In [67]:
liked_q['Score'] = liked_q.apply(weighted_rating, axis=1)

## Best rated (TOP 10)

In [68]:
top_r = liked_q[['Book-Title', 'Rating-Mean']]

In [69]:
top_r = top_r.groupby(['Book-Title'])['Rating-Mean'].mean().to_frame()

In [70]:
top_r.sort_values(by='Rating-Mean', ascending=False).head(10)

Rating-Mean
Book-Title                                                     
The Fellowship of the Ring (The Lord of the Rin...        10.00
The Return of the King (The Lord of the Rings, ...        10.00
The Hobbit : The Enchanting Prelude to The Lord...         9.93
The Two Towers (The Lord of the Rings, Part 2)             9.81
Ender's Game (Ender Wiggins Saga (Paperback))              9.67
Harry Potter and the Goblet of Fire (Book 4)               9.67
Harry Potter and the Order of the Phoenix (Book 5)         9.67
Harry Potter and the Chamber of Secrets (Book 2)           9.29
Harry Potter and the Sorcerer's Stone (Harry Po...         9.17

## Most popular (TOP 10)

In [71]:
top_p = liked[['Book-Title', 'Rating-Count']]

In [72]:
top_p = top_p.groupby(['Book-Title'])['Rating-Count'].mean().to_frame()

In [73]:
top_p.sort_values(by='Rating-Count', ascending=False).head(10)#.plot(kind='barh')

Rating-Count
Book-Title                                                      
The Fellowship of the Ring (The Lord of the Rin...     45.000000
The Return of the King (The Lord of the Rings, ...     22.000000
The Two Towers (The Lord of the Rings, Part 2)         20.090909
The Hobbit : The Enchanting Prelude to The Lord...     15.000000
Harry Potter and the Sorcerer's Stone (Harry Po...      6.000000
Ender's Game (Ender Wiggins Saga (Paperback))           6.000000
Harry Potter and the Order of the Phoenix (Book 5)      6.000000
Harry Potter and the Chamber of Secrets (Book 2)        5.800000
Harry Potter and the Goblet of Fire (Book 4)            5.000000
The Da Vinci Code                                       4.000000

# Embedings (word2vec)

The idea behind this is that different movies are connected via the same user and thus will have high cosine similarity. What this truely shows is how books "travels" - remeber, this is book crossing project. For more information about word2vec see: http://www.bicorner.net/?p=194&preview=true

In [74]:
# ALEX: remove plotting
# from tqdm import tqdm
# from gensim.models import Word2Vec 
import random

In [75]:
users = df["User-ID"].unique().tolist()
len(users)

6310

In [76]:
# shuffle users ID's
random.shuffle(users)

# extract 90% of customer ID's
users_train = [users[i] for i in range(round(0.9*len(users)))]

# split data into train and validation set
train_df = df[df['User-ID'].isin(users_train)]
validation_df = df[~df['User-ID'].isin(users_train)]

In [77]:
# list to capture purchase history of the customers
reads_train = []

# populate the list with the product codes
# ALEX: remove extra display code
# for i in tqdm(users_train):
for i in users_train:
    temp = train_df[train_df["User-ID"] == i]["ISBN"].tolist()
    reads_train.append(temp)

In [78]:
# list to capture purchase history of the customers
reads_val = []

# populate the list with the product codes
# ALEX: remove extra display code
# for i in tqdm(validation_df['User-ID'].unique()):
for i in validation_df['User-ID'].unique():
    temp = validation_df[validation_df["User-ID"] == i]["ISBN"].tolist()
    reads_val.append(temp)

In [79]:
# train word2vec model
# ALEX: remove ML code
# model = Word2Vec(window = 10, sg = 1, hs = 0,
#                  negative = 10, # for negative sampling
#                  alpha=0.03, min_alpha=0.0007,
#                  seed = 14)

# model.build_vocab(reads_train, progress_per=200)

# model.train(reads_train, total_examples = model.corpus_count, 
#             epochs=10, report_delay=1)

In [80]:
# ALEX: remove ML code
# model.init_sims(replace=True)

In [81]:
# ALEX: remove ML code
# print(model)

In [82]:
# extract all vectors
# ALEX: remove ML code
# X = model[model.wv.vocab]

# X.shape

In [83]:
# ALEX: remove ML code, plotting
# import umap

# cluster_embedding = umap.UMAP(n_neighbors=30, min_dist=0.0,
#                               n_components=2, random_state=42).fit_transform(X)

# plt.figure(figsize=(10,9))
# plt.scatter(cluster_embedding[:, 0], cluster_embedding[:, 1], s=3, cmap='Spectral')

In [84]:
books = train_df[["ISBN", "Book-Title"]]

# remove duplicates
books.drop_duplicates(inplace=True, subset='ISBN', keep="last")

# create product-ID and product-description dictionary
books_dict = books.groupby('ISBN')['Book-Title'].apply(list).to_dict()

/tmp/ipykernel_49938/2480998896.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.drop_duplicates(inplace=True, subset='ISBN', keep="last")


In [85]:
# Find LOTR
df[df['Book-Title'].str.contains('Lord of the Rings')].sample()

User-ID        ISBN Book-Rating  Age  \
196974   142672  0618002227          10   30   

                                               Book-Title       Book-Author  \
196974  The Fellowship of the Ring (The Lord of the Ri...  J. R. R. Tolkien   

        Year-Of-Publication                 Publisher  \
196974                 1999  Houghton Mifflin Company   

                                              Image-URL-S  \
196974  http://images.amazon.com/images/P/0618002227.0...   

                                              Image-URL-M  \
196974  http://images.amazon.com/images/P/0618002227.0...   

                                              Image-URL-L  Country  
196974  http://images.amazon.com/images/P/0618002227.0...   canada

In [86]:
# Check
books_dict['0345339703']

['The Fellowship of the Ring (The Lord of the Rings, Part 1)']

In [87]:
# ALEX: remove ML code
# def similar_books(v, n = 15):
    
#     # extract most similar products for the input vector
#     ms = model.similar_by_vector(v, topn= n+1)[1:]
    
#     # extract name and similarity score of the similar products
#     new_ms = []
#     for j in ms:
#         pair = (books_dict[j[0]][0], j[1])
#         new_ms.append(pair)
        
#     return new_ms 

In [88]:
# Recommend
# ALEX: remove ML code
# similar_books(model['0345339703'])